#### Installing dependencies

In [1]:
!pip install numpy torch matplotlib

#### Importing packages

In [2]:
%matplotlib inline
import numpy as np
import torch
import torch.utils.data as Data
from torch.autograd import Variable
import matplotlib.pyplot as plt
from collections import OrderedDict
from math import pi

#### NN class for Regression

In [3]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize, nhiddenLayers, hiddenNeurons):
        super(linearRegression, self).__init__()
        layers = []

        for i in range(0,nhiddenLayers): # adding hidden layers
            if(i==0): #Input layer
                layers.append(torch.nn.Linear(inputSize, hiddenNeurons))
                layers.append(torch.nn.ReLU())
            else:
                layers.append(torch.nn.BatchNorm1d(hiddenNeurons))
                layers.append(torch.nn.Linear(hiddenNeurons, hiddenNeurons))
                layers.append(torch.nn.ReLU())

        layers.append(torch.nn.Linear(hiddenNeurons, outputSize)) # output layer
        self.model = torch.nn.Sequential(*layers)
        print(self.model)

    def forward(self, x):
        out = self.model(x)
        return out

#### NN Parameters

In [4]:
learningRate = 0.001
momentum = 0.9 
epochs = 2000
batch_size = 3000
train_set = 60000
test_set = 60000

#### Making the Dataset

In [5]:
f1 = lambda x: 2 * (x ** 2) - 1
func = lambda x: f1(f1(np.cos(x)))
x = torch.unsqueeze(torch.linspace(-2 * pi, 2 * pi, train_set), dim=1)
y = func(x)

# Train Dataset, ,Making generator for batches
train_dataset = Data.TensorDataset(x, y)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Test Dataset
test_ind = np.random.choice(train_set, test_set)
test_x = x[test_ind]
test_y = func(test_x)

#### Defining Net, Criterion and Optimizer

In [6]:
net = linearRegression(1,1,3, 24)
criterion = torch.nn.MSELoss() # Define criterion to evaluate the network
optimizer = torch.optim.SGD(net.parameters(), lr=learningRate, momentum=momentum)

Sequential(
  (0): Linear(in_features=1, out_features=24, bias=True)
  (1): ReLU()
  (2): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=24, out_features=24, bias=True)
  (4): ReLU()
  (5): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=24, out_features=24, bias=True)
  (7): ReLU()
  (8): Linear(in_features=24, out_features=1, bias=True)
)


#### Training the NN

In [9]:
for epoch in range(epochs): # Iteration in each epoch
    for step, (t_x, t_y) in enumerate(train_loader): # Iterating in each batch
        optimizer.zero_grad() # Removing buffer from previous epochs
        outputs = net(t_x) # Output training into the model
        loss = criterion(outputs, t_y) # Get loss for predicted outputs
        if(epoch%10 == 0 and step == train_set/batch_size-1):
            print("Epoch: ", epoch,"  Step: ", step,"  Loss: ", loss)
        loss.backward() # Propagate the loss
        optimizer.step() # Update parameters

Epoch:  0   Step:  19   Loss:  tensor(0.0167, grad_fn=<MseLossBackward>)
Epoch:  10   Step:  19   Loss:  tensor(0.0024, grad_fn=<MseLossBackward>)
Epoch:  20   Step:  19   Loss:  tensor(0.0020, grad_fn=<MseLossBackward>)
Epoch:  30   Step:  19   Loss:  tensor(0.0037, grad_fn=<MseLossBackward>)
Epoch:  40   Step:  19   Loss:  tensor(0.0042, grad_fn=<MseLossBackward>)
Epoch:  50   Step:  19   Loss:  tensor(0.0054, grad_fn=<MseLossBackward>)
Epoch:  60   Step:  19   Loss:  tensor(0.0025, grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

#### Evaluating the model

In [ ]:
#net = net.eval()
test_outputs = net(test_x)
mean_test_error = torch.mean(torch.abs(test_y-test_outputs))
print(torch.abs(test_y-test_outputs))
print("Mean Test Error: ", mean_test_error)